In [1]:
from sklearn import metrics, model_selection, ensemble
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
import xgboost as xgb
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import StackingRegressor
import xgboost
from sklearn.cluster import KMeans


pd.set_option('display.max_columns', None)

C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
try: 
    import optuna
except:
    !pip install optuna
    import optuna

In [3]:
try:
    from catboost import Pool, CatBoostRegressor, cv
except:
    !pip install catboost 
    from catboost import Pool, CatBoostRegressor, cv

In [4]:
try:
    from CustomPipeline import *
except:
    import sys
    sys.path.insert(0,'/content/sample_data')
    from CustomPipeline import *
    print("ok")

In [114]:
try:
    train = pd.read_csv("./train_anomaly.csv", delimiter=",", sep='.')
except:
    train = pd.read_csv("./sample_data/train_anomaly.csv", delimiter=",", sep='.')

In [115]:
RANDOM_STATE = 42

In [116]:
train.drop("id", axis=1, inplace=True)
train["target"] = 100 * train["target"]

In [117]:
# test run
# train, test = train_test_split(train, test_size=0.2, random_state=RANDOM_STATE)

In [118]:
train, test_ensemble = train_test_split(train, test_size=0.1, random_state=RANDOM_STATE)
train_estimators, train_ensemble = train_test_split(train, test_size=0.2, random_state=RANDOM_STATE)

In [119]:
X = train_estimators.drop(["target"], axis=1)
y = train_estimators["target"]

In [120]:
num = TypesOfColumns(X).get_num()
cat = TypesOfColumns(X).get_cat()
cat_idx = TypesOfColumns(X).get_cat_idx()

# linear ridge

In [121]:
params_linear = {'alpha': 0.09}
n_bins=63

X_lr = train_estimators.query("target > 600").drop(["target"], axis=1)
y_lr = train_estimators.query("target > 600")["target"]

model = linear_model.Ridge(**params_linear)
lr = LinearWrapper(model, bins_linear=n_bins, cat=cat, num=num)

TypeError: __init__() got an unexpected keyword argument 'cat'

# xgb

In [ ]:
params_xgb = {
    'tree_method':'gpu_hist',
    'random_state': 1, 
    'n_jobs': 4,
    'booster': 'gbtree',
    'n_estimators': 10000,
    'learning_rate': 0.035,
    'reg_lambda': 1.22,
    'reg_alpha': 36.04,
    'subsample': 0.9,
    'colsample_bytree': 0.11,
    'max_depth': 3,
    'min_child_weight': 6
}
model = xgboost.XGBRegressor(**params_xgb)
xgb = XGBWrapper(model, cat=cat, num=num)

# StackingRegressor

In [ ]:
# test = pd.read_csv("./sample_data/test_anomaly.csv", delimiter=",", sep='.')

In [ ]:
models = [lr, xgb]
# models = [catboost]

df_ensemble = pd.DataFrame()
df_test = pd.DataFrame()
df_submit = pd.DataFrame()

for i, model in enumerate(models):
    if model == lr:
        model.train(X_lr, y_lr)
    else:
        model.train(X, y)

    pred = model.predict(train_ensemble.drop(['target'], axis=1))
#     print(pred)
    train_ensemble[str(i)] = pred
    pred = model.predict(test_ensemble.drop(['target'], axis=1))
#     print(pred)
    test_ensemble[str(i)] = pred
    pred = model.predict(test)
#     print(pred)
    test[str(i)] = pred

In [ ]:
# train_ensemble.to_csv('./train_ensemble.csv', index=False)
# test_ensemble.to_csv('./test_ensemble.csv', index=False)
# test.to_csv('./test.csv', index=False)

In [12]:
test_ensemble = pd.read_csv("./en_test_ensemble.csv", delimiter=",", sep='.')
test = pd.read_csv("./en_test_witout_target.csv", delimiter=",", sep='.', index_col='id')
train_ensemble = pd.read_csv("./en_train_ensemble.csv", delimiter=",", sep='.')

In [123]:
test['label'] = 'test'
train_ensemble['label'] = 'train_ensemble'
test_ensemble['label'] = 'test_ensemble'

In [124]:
df = pd.concat([test, train_ensemble.drop('target', axis=1), test_ensemble.drop('target', axis=1)])

In [125]:
X = df
num_train = X.select_dtypes([int, float])
cat_train = X.select_dtypes(object)

num = list(num_train)
cat = list(cat_train)

In [ ]:
for i, m in enumerate(num):
    for j, n in enumerate(num):
        if (m != n) and (j > i):
            print(m, n)
            try:
                kmeans = KMeans(n_clusters=9, random_state=42).fit(df[[m, n]])
                df['k_means_' + m + n] = kmeans.labels_
                df['k_means_' + m + n].astype(object)
            except:
                print(m, n, 'problem')


cont0 cont1
cont0 cont2
cont0 cont3
cont0 cont4
cont0 cont5
cont0 cont6
cont0 cont7
cont0 cont8
cont0 cont9
cont0 cont10
cont0 cont11
cont0 cont12
cont0 cont13
cont0 0
cont0 1
cont1 cont2
cont1 cont3
cont1 cont4
cont1 cont5
cont1 cont6
cont1 cont7
cont1 cont8
cont1 cont9
cont1 cont10
cont1 cont11
cont1 cont12
cont1 cont13
cont1 0
cont1 1
cont2 cont3
cont2 cont4
cont2 cont5
cont2 cont6
cont2 cont7
cont2 cont8
cont2 cont9
cont2 cont10
cont2 cont11
cont2 cont12
cont2 cont13
cont2 0
cont2 1
cont3 cont4
cont3 cont5
cont3 cont6
cont3 cont7
cont3 cont8
cont3 cont9
cont3 cont10
cont3 cont11
cont3 cont12
cont3 cont13
cont3 0
cont3 1
cont4 cont5
cont4 cont6
cont4 cont7
cont4 cont8
cont4 cont9
cont4 cont10
cont4 cont11
cont4 cont12
cont4 cont13
cont4 0
cont4 1
cont5 cont6
cont5 cont7
cont5 cont8
cont5 cont9
cont5 cont10
cont5 cont11
cont5 cont12
cont5 cont13
cont5 0
cont5 1
cont6 cont7
cont6 cont8
cont6 cont9
cont6 cont10
cont6 cont11
cont6 cont12
cont6 cont13
cont6 0
cont6 1
cont7 cont8
cont7 co

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_fast._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 2, in where
KeyboardInterrupt


cont8 cont11
cont8 cont12
cont8 cont13
cont8 cont13 problem
cont8 0


In [ ]:
df['k_means_cont4_cont6_'] = df['cont4'].apply(lambda x: 1 if x<0.1 else 0)
df['k_means_cont4_cont6_'].astype(object)

df['k_means_cont5_cont8_'] = df['cont8'].apply(lambda x: 1 if x<0.1 else 0)
df['k_means_cont5_cont8_'].astype(object)

df['k_means_cont6_cont7_'] = df['cont6'].apply(lambda x: 1 if x<0.1 else 0)
df['k_means_cont6_cont7_'].astype(object)

In [ ]:
df.to_csv('./ensemble_with_clusters.csv', index=False)

In [6]:
df = pd.read_csv("./ensemble_with_clusters.csv", delimiter=",", sep='.')

In [32]:
df.iloc[:, 28:] = df.iloc[:, 28:].astype('object')

In [33]:
test_new = df[df['label'] == 'test']
train_ensemble_new = df[df['label'] == 'train_ensemble']
test_ensemble_new = df[df['label'] == 'test_ensemble']

In [35]:
X = train_ensemble_new.drop('label', axis=1)
y = train_ensemble['target']

num_train = X.select_dtypes([int, float])
cat_train = X.select_dtypes(object)

num = list(num_train)
cat = list(cat_train)

In [16]:
X = train_ensemble_new.drop('label', axis=1)
y = train_ensemble['target']

num_train = X.select_dtypes([int, float])
cat_train = X.select_dtypes(object)

num = list(num_train)
cat = list(cat_train)

rmse = make_scorer(mean_squared_error, squared=False)

pipeline_num = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaling', StandardScaler()),  
    ('normal', PowerTransformer()), 
])
pipeline_cat = Pipeline(steps=[
    # ('imputer', SimpleImputer(strategy='most_frequent')),
    # ('encoding', OrdinalEncoder()),
    ('encoding', OneHotEncoder(handle_unknown='ignore')),
])
preprocessor = ColumnTransformer(n_jobs=-1,
    transformers=[
        ('num', pipeline_num, num),
        ('cat', pipeline_cat, cat),
        ], remainder="passthrough")

preprocessor.fit(df.drop('label', axis=1))
transform = preprocessor.transform(X)
X_test_ensemble = preprocessor.transform(test_ensemble_new.drop('label', axis=1))
X_test = preprocessor.transform(test_new.drop('label', axis=1))

In [17]:
def objective(trial):
    
    param_model = {
        'alpha': trial.suggest_loguniform('alpha', 1e-2, 1.0),
        'random_state':trial.suggest_categorical("random_state", [0, 42]),
    }
    
    pipeline_ridge = Pipeline(steps=[ 
                                     ('model', linear_model.Lasso(**param_model)),
                                     ])

    rmse_mean_cv = cross_val_score(pipeline_ridge, transform, y, cv=5, scoring=rmse)
    print(rmse_mean_cv)

    return rmse_mean_cv.mean()

In [18]:
study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=10)
# 72.05929420329886.
# 72.05702112640415.

[I 2021-08-30 20:17:09,107] A new study created in memory with name: no-name-35e10616-9553-4b80-a83a-cbf8b18026d0
[I 2021-08-30 20:17:19,840] Trial 0 finished with value: 72.27976343151954 and parameters: {'alpha': 0.2554627245885052, 'random_state': 0}. Best is trial 0 with value: 72.27976343151954.


[72.55813781 72.49766008 71.95411253 72.14743191 72.24147482]


[I 2021-08-30 20:17:28,130] Trial 1 finished with value: 72.28349862244272 and parameters: {'alpha': 0.6034975317104673, 'random_state': 0}. Best is trial 0 with value: 72.27976343151954.


[72.58368312 72.52428158 71.9244071  72.13243511 72.25268619]


[I 2021-08-30 20:17:41,934] Trial 2 finished with value: 72.28235577929613 and parameters: {'alpha': 0.21538624734626244, 'random_state': 0}. Best is trial 0 with value: 72.27976343151954.


...

[I 2021-08-30 20:20:51,302] Trial 6 finished with value: 72.28477863550206 and parameters: {'alpha': 0.18754860047869457, 'random_state': 0}. Best is trial 0 with value: 72.27976343151954.


[72.55616486 72.4966627  71.96580446 72.1606284  72.24463276]


[I 2021-08-30 20:21:00,997] Trial 7 finished with value: 72.27451990177444 and parameters: {'alpha': 0.40683429928902226, 'random_state': 0}. Best is trial 7 with value: 72.27451990177444.


[72.56499299 72.50271514 71.93289444 72.13265573 72.23934121]


KeyboardInterrupt: 

In [ ]:
best_params = study.best_trial.params
best_params 

# check bias clusters

In [ ]:
from sklearn.linear_model import Lasso
model = Lasso(**best_params)
model.fit(transform, y)

In [ ]:
model.coef_

In [ ]:
pred = model.predict(transform)
print(mean_squared_error(train_ensemble["target"], pred, squared=False))
train_ensemble['pred_lasso'] = pred

In [ ]:
pred = model.predict(X_test_ensemble)
print(mean_squared_error(test_ensemble["target"], pred, squared=False))

In [ ]:
pred = model.predict(X_test)
test['target'] = pred / 100

In [ ]:
train_ensemble

In [ ]:
train_ensemble['residuals'] = train_ensemble['target'] - train_ensemble['0']
train_ensemble['label'] = train_ensemble.residuals.apply(lambda x: 1 if x > -180 else 0) 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
residuals = train_ensemble['target'] - train_ensemble['0']
sns.scatterplot(train_ensemble['0'], residuals, hue = train_ensemble['label'], size=train_ensemble['label'])

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
residuals = train_ensemble['target'] - train_ensemble['pred_lasso']
sns.scatterplot(train_ensemble['pred_lasso'], residuals, hue = train_ensemble['label'], size=train_ensemble['label'])

In [ ]:
# for i, m in enumerate(num):
#     for j, n in enumerate(num):
#         if (m != n) and (j > i):
#             plt.figure()
#             sns.scatterplot(train_ensemble[m], train_ensemble[n], hue = train_ensemble['label'], size=train_ensemble['label'])

# save predict

In [ ]:
# test = pd.read_csv("./sample_data/test_anomaly.csv", delimiter=",", sep='.')

In [ ]:
# df_ensemble = pd.DataFrame()
# df_ensemble['lr'] = lr_pred
# df_ensemble['xgb_pred'] = xgb_pred
# df_ensemble['cat_pred'] = cat_pred

# df_test = pd.DataFrame()
# df_test['lr'] = lr_test
# df_test['xgb_pred'] = xgb_test
# df_test['cat_pred'] = cat_test

In [ ]:
# # del
# lr_pred1 = lr.predict(test)
# xgb_pred1 = xgb.predict(test)
# cat_pred1 = catboost.predict(test)

# df_submit = pd.DataFrame()
# df_submit['lr'] = lr_pred1
# df_submit['xgb_pred'] = xgb_pred1
# df_submit['cat_pred'] = cat_pred1

In [ ]:
# pred = pipeline_ridge.predict(test)

In [ ]:
test.head()

In [ ]:
test[['target']].to_csv('./ensemble_for_submit_with_clusters.csv', index=True)